## Import Required Libraries
Running cell below will import the required libraries being used in the rest of the cells of this notebook

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import math
import os
from datetime import datetime
pd.set_option('display.max_columns', None)

from matplotlib import rc
rc('text',usetex=True)
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
rc('text.latex', preamble='\\usepackage{color} \\usepackage{amsmath} \\usepackage{amssymb}')

import warnings
warnings.filterwarnings("ignore")

# Select and load the CSV

In [ ]:
exp_type ='E4'

## Select model_type

In [ ]:
# model_name    = 'yolov8s'
model_name    = 'yolo11s'

## Select background_type

In [ ]:
background_type  = 'gray'     # black , white , gray , noise , blurred , full
# background_type  = 'black'     # black , white , gray , noise , blurred , full
# background_type  = 'white'     # black , white , gray , noise , blurred , full
# background_type  = 'noise'     # black , white , gray , noise , blurred , full
# background_type  = 'blurred'     # black , white , gray , noise , blurred , full
# background_type  = 'full'     # black , white , gray , noise , blurred , full

## XAI Methods ?

In [ ]:
# num_xai_methods = 5
# num_xai_methods = 7     # AAx3 , BPTx3, LIMEx1  
num_xai_methods = 9     # AAx3 , BPTx3, LIMEx3  -> LATEST

## Target Num Images

In [ ]:
num_images_target = 5000

# Path Setting


In [ ]:
def get_file_modify_time(filepath=None):
    if os.path.exists(filepath):
        datestamp  = datetime.fromtimestamp(os.path.getmtime(filepath))
        print('File Exists :', filepath)
        print('File Date/Time :', datestamp)

In [ ]:
suffix      = f'{model_name}_{background_type}'
codes_path    = os.getcwd() 
# path_csv          = os.path.join(codes_path,     'csv')
path_csv    = '../results_logits_with_auc_clipped/'
path_boxplot          = os.path.join(codes_path,     'boxplot')
os.makedirs(path_boxplot,              exist_ok=True)
print('path_csv :',path_csv)
print('path_boxplot :',path_boxplot)


In [ ]:
# # suffix_full      = f'{model_type}_{background_type}'

# # if select_random_test:
# #     subset_type = 'im50'
# #     # subset_type = 'im300'
    
# path_notebook = os.path.abspath(os.path.join(os.getcwd(), os.pardir))


# codes_path    = os.getcwd() 
# results_path    = os.path.join(codes_path,'results')

# if model_type=='real':
#     results_path    = os.path.join(results_path,pretrained_model_type)
#     suffix                     = f'{model_name}_{background_type}'
# else:
#     results_path    = os.path.join(results_path)
#     suffix                     = f'{model_name}_{background_type}'


# results_path    = os.path.join(results_path,suffix)

# path_csv          = os.path.join(codes_path,     'csv')
# path_boxplot          = os.path.join(codes_path,     'boxplot')
# os.makedirs(path_boxplot,              exist_ok=True)

In [ ]:
csv_filename = f'{path_csv}/csv_exp_E4_{suffix}_{num_xai_methods}.csv'

get_file_modify_time(filepath = csv_filename)
if os.path.exists(csv_filename):
    print('csv file ', csv_filename)
    df = pd.read_csv(csv_filename)
else:
    print('csv file NOT FOUND', csv_filename)

In [ ]:
df.rename(columns={'auc_IoU': 'au_IoU'}, inplace=True)

In [ ]:
df.method = df.method.apply(lambda x: x.replace('Partition', 'AA'))
df.method = df.method.apply(lambda x: x.replace('ShapGradE', 'GradShap'))

df.head(3)

In [ ]:
def inspect_df(df):
    computed_methods = df.method.unique()
    num_methods = len(computed_methods)
    num_images  = len(np.unique(df.image))
    return computed_methods,num_methods,num_images

In [ ]:
computed_methods,num_methods,num_images = inspect_df(df)
print(num_methods,computed_methods)
print('-'*120)
print(f'Total Images: {num_images}')

In [ ]:
# df.method = df.method.apply(lambda x: x.replace('LIME-50', 'LIME-250'))
# df.method = df.method.apply(lambda x: x.replace('LIME-100', 'LIME-500'))
# df.method = df.method.apply(lambda x: x.replace('LIME-200', 'LIME-1000'))

# computed_methods,num_methods,num_images = inspect_df(df)
# print(num_methods,computed_methods)
# print('-'*120)
# print(f'Total Images: {num_images}')

In [ ]:
print(f'Total Images: {num_images}')

In [ ]:
print('model_name:\t\t', model_name)
print('background_type:\t', background_type)

In [ ]:
# def get_methods_symbols():
#     pass
# def get_name_remap():
#     pass
# def get_method_remap():
#     pass
# def get_method_colors():
#     pass

In [ ]:
name_remap = {
    'Partition-100': 'PE-100',
    'Partition-500': 'PE-500',
    'Partition-1000': 'PE-1000',
    'BPT-100': '\\bf BPT-100',
    'BPT-500': '\\bf BPT-500',
    'BPT-1000': '\\bf BPT-1000',
    'aIDG': '$|$IDG$|$',
    'aGradExpl': '$|$GradExpl$|$',
}
method_remap = {
    'max_IoU': '$max$-$IoU$',
    'au_IoU': '$AU$-$IoU$',
    'aucI_pred': '$\\mathit{AUC}^+$',
    'aucD_pred': '$\\mathit{AUC}^-$',
    'aucI_mse': '$\\mathit{MSE}^+$',
    'aucD_mse': '$\\mathit{MSE}^-$',
    'time_exp': 'log(time)'
}
method_colors = {
    'BPT-100':         '#f2a7c0',
    'BPT-500':         '#db587f',
    'BPT-1000':        '#9d2f4d',
    'AA-100':   '#4ed1f9',
    'AA-500':   '#00b1f2',
    'AA-1000':  '#008fe8',
    'AA-5000':  '#006bd7',
    'AA-10000':  '#3245bd',
    'LIME-100':         '#c7e883',
    'LIME-500':        '#8fc543',
    'LIME-1000':        '#66912d',
    'LRP':        '#008080',
    'GradCAM':         'xkcd:camel',
    'aIDG':            '#fcd459',
    'aGradExpl':       '#dd7c2b',
    'GradShap':       '#800000',
}
method_symbol = {
    'BPT-100'   :       '$\\clubsuit$1',
    'BPT-500'   :       '$\\clubsuit$2',
    'BPT-1000'  :       '$\\clubsuit$3',
    'AA-100'    :       '$\\boxplus$1',
    'AA-500'    :       '$\\boxplus$2',
    'AA-1000'   :       '$\\boxplus$3',
    'AA-5000'   :       '$\\boxplus$4',
    'AA-10000'  :       '$\\boxplus$5',
    'LIME-100'   :       '\\textbf{0}1',
    'LIME-500'  :      '\\textbf{0}2',
    'LIME-1000'  :      '\\textbf{0}3',
    'LRP'       :      '\\textbf{$\\Pi$}',
    'GradCAM'   :      '$\\bigtriangledown$',
    'aIDG'      :      '$\\divideontimes$',
    'aGradExpl' :     '$\\bigtriangleup$',
     'GradShap':       '$\\square$',
}

method_to_shortname = {}
for i,m in enumerate(method_colors.keys()):
    method_to_shortname[m] = f'{method_symbol[m]}'
    print(f'{m:15} : {method_to_shortname[m]}')

In [ ]:
for m in df.method.unique():
    if m not in method_colors:
        print(f'WARNING: {m} not in method_colors')
        continue
    if m not in method_to_shortname:
        print(f'WARNING: {m} not in method_to_shortname')
        continue

print('='*120)
for m in method_colors:
    if m not in df.method.unique():
        print(f'WARNING: {m} not in df.method.unique()')
        continue
    if m not in method_to_shortname:
        print(f'WARNING: {m} not in method_to_shortname')
        continue

# Plot Results

In [ ]:
def get_header(model_name,background_type,num_xai_methods,df):
    computed_methods = np.unique(df.method)
    num_methods = len(computed_methods)
    computed_images  = len(np.unique(df.image))
    print('='*96)
    print('| ',f'{"model_name":15}','|',f'{"BACKGROUND_TYPE":18}','|',f'{"target_methods":<15}','|',f'{"computed_images":15}','|',f'{"computed_methods":15}','|')
    print('-'*96)
    print('| ',f'{model_name:15}','|',f'{background_type:18}','|',f'{num_xai_methods:<15}','|',f'{computed_images:15}','|',f'{num_methods:15}','|')

get_header(model_name,background_type,num_xai_methods,df)

## Figure 4: E1, E2

In [ ]:
filename_boxplots = f'{path_boxplot}/results_E4_{suffix}_{num_methods}'
filename_boxplots

In [ ]:
method_remap = {
    'max_IoU': '$max$-$IoU$',
    'au_IoU': '$AU$-$IoU$',
    'aucI_pred': '$\\mathit{AUC}^+$',
    'aucD_pred': '$\\mathit{AUC}^-$',
    'aucI_adj': '$\\mathit{AUC}^+-Adju$',
    'aucD_adj': '$\\mathit{AUC}^--Adju$',
    
    'aucI_r': '$\\mathit{AUC}^+-R$',
    'aucD_r': '$\\mathit{AUC}^--R$',

    'aucI_adj_r': '$\\mathit{AUC}^+-Adju-R$',
    'aucD_adj_r': '$\\mathit{AUC}^--Adju-R$',

    'aucI_clip': '$\\mathit{AUC}^+-Cl$',
    'aucI_clipr': '$\\mathit{AUC}^+-Cl-R$',

    'aucD_clip': '$\\mathit{AUC}^--Cl$',
    'aucD_clipr': '$\\mathit{AUC}^--Cl-R$',

    'time_exp': 'log(time)'
}

arrow_up = '$\\Uparrow$'
arrow_down = '$\\Downarrow$'
arrow = {

    'max_IoU': arrow_up,
    'au_IoU': arrow_up,
    'aucI_clipr': arrow_up,
    
    'aucD_clipr': arrow_down,
    'time_exp': arrow_down,
}

In [ ]:
# from datetime import datetime
import datetime
def box_plot(ax, data, label, position, edge_color, line_width, fill_color):
    bp = ax.boxplot(data,
                    # labels=[label],
                    tick_labels=[label],
                    positions=[position], vert=False, 
                    widths=0.5, whis=5, patch_artist=True, flierprops={'markersize': 1, 'markerfacecolor':'magenta', 'markeredgecolor':'magenta'}) # 
    
    for element in ['boxes', 'whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color=edge_color, linewidth=line_width)

    for patch in bp['boxes']:
        patch.set(facecolor=fill_color)       
        
    return bp


# auc_methods      = ['aucI_pred', 'aucD_pred', 'aucI_adj', 'aucD_adj', 'aucI_r','aucD_r',\
#                     'aucI_adj_r','aucD_adj_r', 'max_IoU', 'au_IoU', 'time_exp']

auc_methods      = ['aucI_pred','aucI_clip' , 'aucI_adj' , 'aucI_r', 'aucI_adj_r', 'max_IoU','time_exp',\
                    'aucD_pred','aucD_clipr', 'aucD_adj' , 'aucD_r', 'aucD_adj_r','au_IoU',\
                    ]

auc_method_ascending = ['aucI_pred','aucI_clip','aucI_adj','aucI_r', 'aucI_adj_r''max_IoU', 'au_IoU' ]

total_images = len(df.image.unique())
total_methods = len(df.method.unique())
ttl_cols = len(auc_methods) if len(auc_methods)%2==0 else len(auc_methods)+1 


# print(f'Explained Class: {explained_class_final}')
print(f'Total Images: {total_images}')
print(f'XAI Methods: {total_methods}')
# print('Background type:\t',background_type)

print('Total Time: \t\t',str(datetime.timedelta(seconds=np.sum(df.time_total))))

fig, axes = plt.subplots(2, int(ttl_cols/2), figsize=(15, 7))  # (rows, cols)
for j, score in enumerate(auc_methods):
    row, col = divmod(j, int(ttl_cols/2))
    ax = axes[row, col]
    df = df[~df[score].isna()] 
    # df = df
    ascending = score in auc_method_ascending
    means = df.groupby('method')[[score]].median() \
              .sort_values(score, ascending=ascending).index
    log_scale = (score=='time_exp')
    ax.set_xscale('log' if log_scale else 'linear')
    for i, method in enumerate(means):
        text_yincrement = -0.3
        values = df[ df.method==method ][score]
        name = name_remap[method] if method in name_remap else method
        color = method_colors[method]
        q75 = np.quantile(values, q=0.75)
        q100 = np.quantile(values, q=1.0)
        if not log_scale and 'IoU' not in score and q100>1.0:
            q100, text_yincrement = 0.82, 0.15
        val_arr = [[v for v in values]]
        box_plot(ax, val_arr, '', i, 
                 'blue' if method.startswith('BPT') else 'black', 
                 1.5 if method.startswith('BPT') else 0.5,
                 color)
        # print(method,q100,score,np.max(values))
        if score in ['aucI_pred', 'aucD_pred']:
            ax.text(min(q100*1.3, 30) if log_scale else q75+0.5, i+text_yincrement, method_to_shortname[method])
        elif score in ['aucI_mse', 'aucD_mse']:
            ax.text(min(q100*1.3, 30) if log_scale else q100+60, i+text_yincrement, method_to_shortname[method])
        else:
            ax.text(min(q100*1.3, 30) if log_scale else q100+0.05, i+text_yincrement, method_to_shortname[method])
    values = df[score]

    ##########################################
    # if score=='aucI_pred': min_x,max_x=  0,1.0 #-np.quantile(values,q=0.1),np.quantile(values,q=1)+(np.quantile(values,q=0)/np.quantile(values,q=1)*45)
    # if score=='aucI_pred': min_x,max_x= 0.01, 10

    
    
    if score=='aucI_r' : min_x,max_x= 0,1
    
    if score=='aucI_adj_r' : min_x,max_x= 0,1.5
    if score=='aucI_mse' : min_x,max_x= -np.quantile(values,q=0.01),400
    if score=='aucI_mse' : min_x,max_x= -np.quantile(values,q=0.01),400

    if score=='aucD_mse' : min_x,max_x= -np.quantile(values,q=0.01),400
    if score=='aucD_pred': min_x,max_x= 0,np.quantile(values,q=1)+(np.quantile(values,q=0)/np.quantile(values,q=1)*20)
    
    if score=='max_IoU'  : min_x,max_x= -np.quantile(values,q=0.01),np.quantile(values,q=1)+(np.quantile(values,q=0)/np.quantile(values,q=1)*20)
    if score=='au_IoU'   : min_x,max_x= -np.quantile(values,q=0.01),np.quantile(values,q=1)+(np.quantile(values,q=0)/np.quantile(values,q=1)*30)
    ##########################################
    # print(score,min_x, max_x)
    # if score=='max_IoU':
    #     print(min_x,max_x)
    if score=='time_exp' :
        min_x, max_x = 0.011, 99
        for m in [0.01, 0.1, 1, 10]:
            for k in range(1,10):
                ax.axvline(m*k, lw=0.5, color='grey', alpha=1/k)
        ax.set_xticks([0.1, 1, 10])
        ax.set_xticklabels(['0.1', '1', '10'])
    if score=='aucI_r' : 
        min_x,max_x= 0,1.5
        ax.set_xlim(min_x, max_x)
    # ax.set_xlim(min_x, max_x)
    ax.set_title(f"{method_remap[score]}", fontsize=15)

    ax.set_ylim(-0.7, i+0.8)
    ax.set_yticks([])

plt.subplots_adjust(wspace=0.10)
# filename_box_plot = f'{results_boxplots}/results_table_IoU_{len(np.unique(df.image))}_{len(methods)}_{background_type}'

plt.savefig(f'{filename_boxplots}.pdf', dpi=150, transparent=True, bbox_inches='tight', pad_inches=0.02)
plt.savefig(f'{filename_boxplots}.svg', dpi=150, transparent=True, bbox_inches='tight', pad_inches=0.02)
# plt.savefig(f'{filename_box_plot}.png', dpi=150, transparent=True, bbox_inches='tight', pad_inches=0.02)


################################
from matplotlib.patches import Patch

last_ax = axes[-1, -1]  # bottom-right subplot

# Unique method list in the order you want
lm = list(df.method.unique())

# Create legend handles with real name + symbol
legend_handles = []
for m in lm:
    shortname = method_to_shortname.get(m, m)
    fullname = name_remap.get(m, m)
    label = f'{shortname}: {fullname}'
    patch = Patch(
        facecolor=method_colors[m],
        edgecolor='blue' if m.startswith('BPT') else 'black',
        linewidth=1.5 if m.startswith('BPT') else 0.5,
        label=label
    )
    legend_handles.append(patch)
for ti, tl in zip(('Total Images','XAI Methods'),(total_images,total_methods)):
    summary_patch = Patch(facecolor='white', edgecolor='white', label=[ti,tl])
    legend_handles.append(summary_patch)

# Add legend to bottom-right subplot (split in 2 columns)
last_ax.legend(
    handles=legend_handles,
    ncol=1,
    loc='center left',
    fontsize=9.5,
    frameon=False
)
plt.suptitle(f'[Experiment:{exp_type}_{model_name} ] --- [Total Images : {total_images}] --- [total_methods : {total_methods}]\
             --- [Total Time: {str(datetime.timedelta(seconds=np.sum(df.time_total)))}]', fontsize=16)
plt.show()
# print('file saved at :', filename_box_plot)

## FOR PAPER

In [ ]:
method_remap = {
    'max_IoU': '$max$-$IoU$',
    'au_IoU': '$AU$-$IoU$',

    'aucI_clipr': '$\\mathit{AUC}^+$',
    'aucD_clipr': '$\\mathit{AUC}^-$',

    'time_exp': 'log(time)'
}

arrow_up = '$\\Uparrow$'
arrow_down = '$\\Downarrow$'
arrow = {

    'max_IoU': arrow_up,
    'au_IoU': arrow_up,
    'aucI_clipr': arrow_up,
    
    'aucD_clipr': arrow_down,
    'time_exp': arrow_down,
}

In [ ]:
# from datetime import datetime
import datetime
def box_plot(ax, data, label, position, edge_color, line_width, fill_color):
    bp = ax.boxplot(data,
                    # labels=[label],
                    tick_labels=[label],
                    positions=[position], vert=False, 
                    widths=0.5, whis=5, patch_artist=True, flierprops={'markersize': 1, 'markerfacecolor':'magenta', 'markeredgecolor':'magenta'}) # 
    
    for element in ['boxes', 'whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color=edge_color, linewidth=line_width)

    for patch in bp['boxes']:
        patch.set(facecolor=fill_color)       
        
    return bp


# auc_methods      = ['aucI_pred', 'aucD_pred', 'aucI_adj', 'aucD_adj', 'aucI_r','aucD_r',\
#                     'aucI_adj_r','aucD_adj_r', 'max_IoU', 'au_IoU', 'time_exp']


auc_methods      = [
                    # 'aucI_pred','aucD_pred',\
                    'aucI_clipr' ,'aucD_clipr', \
                    'max_IoU','au_IoU',\
                    'time_exp']

auc_method_ascending = ['aucI_clipr','aucI_clip','aucI_adj','aucI_r', 'aucI_adj_r','max_IoU', 'au_IoU' ]

total_images = len(df.image.unique())
total_methods = len(df.method.unique())
ttl_cols = len(auc_methods) if len(auc_methods)%2==0 else len(auc_methods)+1 


# print(f'Explained Class: {explained_class_final}')
print(f'Total Images: {total_images}')
print(f'XAI Methods: {total_methods}')
# print('Background type:\t',background_type)

print('Total Time: \t\t',str(datetime.timedelta(seconds=np.sum(df.time_total))))

fig, axes = plt.subplots(1, len(auc_methods), figsize=(12, 2.5))  # (rows, cols)
for j, score in enumerate(auc_methods):
    # row, col = divmod(j, int(ttl_cols/2))
    ax = axes[j]
    # df = df[~df[score].isna()] 
    # df = df
    ascending = score in auc_method_ascending
    means = df.groupby('method')[[score]].median() \
              .sort_values(score, ascending=ascending).index
    log_scale = (score=='time_exp')
    ax.set_xscale('log' if log_scale else 'linear')
    for i, method in enumerate(means):
        text_yincrement = -0.3
        values = df[ df.method==method ][score]
        values = values[~values.isna()]
        name = name_remap[method] if method in name_remap else method
        color = method_colors[method]
        q75 = np.quantile(values, q=0.75)
        q100 = np.quantile(values, q=1.0)
        if not log_scale and 'IoU' not in score and q100>1.0:
            q100, text_yincrement = 0.82, 0.15
        val_arr = [[v for v in values]]
        box_plot(ax, val_arr, '', i, 
                 'blue' if method.startswith('BPT') else 'black', 
                 1.5 if method.startswith('BPT') else 0.5,
                 color)
        # print(method,q100,score,np.max(values))
        if score in ['aucI_pred', 'aucD_pred']:
            ax.text(min(q100*1.3, 30) if log_scale else q75+0.5, i+text_yincrement, method_to_shortname[method])
        elif score in ['aucI_mse', 'aucD_mse']:
            ax.text(min(q100*1.3, 30) if log_scale else q100+60, i+text_yincrement, method_to_shortname[method])
        else:
            ax.text(min(q100*1.3, 30) if log_scale else q100+0.05, i+text_yincrement, method_to_shortname[method])
    values = df[score]

    ##########################################
    # if score=='aucI_pred': min_x,max_x=  0,1.0 #-np.quantile(values,q=0.1),np.quantile(values,q=1)+(np.quantile(values,q=0)/np.quantile(values,q=1)*45)
    # if score=='aucI_pred': min_x,max_x= 0.01, 10

    
    
    if score=='aucI_r' : min_x,max_x= 0,1
    
    if score=='aucI_clipr' : min_x,max_x= -0.05,1.2
    if score=='aucD_clipr' : min_x,max_x= -0.05,1.3
    # if score=='aucI_mse' : min_x,max_x= -np.quantile(values,q=0.01),400
    # if score=='aucI_mse' : min_x,max_x= -np.quantile(values,q=0.01),400
    
    if score=='max_IoU'  : min_x,max_x= -0.05, 1.2 #-np.quantile(values,q=0.01),np.quantile(values,q=1)+(np.quantile(values,q=0)/np.quantile(values,q=1)*20)
    if score=='au_IoU'   : min_x,max_x= -0.05, 0.7 #-np.quantile(values,q=0.01),np.quantile(values,q=1)+(np.quantile(values,q=0)/np.quantile(values,q=1)*30)
    ##########################################
    # print(score,min_x, max_x)
    # if score=='max_IoU':
    #     print(min_x,max_x)
    if score=='time_exp' :
        min_x, max_x = 1, 99
        for m in [1, 20]:
            for k in range(1,20):
                ax.axvline(m*k, lw=0.5, color='grey', alpha=1/k)
        ax.set_xticks([1,5, 20])
        ax.set_xticklabels(['1', '5','20'])
    # if score==', maucI_r' : 
    #     min_x,max_x= 0,1.5
    #     ax.set_xlim(min_xax_x)
    ax.set_xlim(min_x, max_x)
    ax.set_title(f"{arrow[score]} {method_remap[score]}", fontsize=15)

    ax.set_ylim(-0.7, i+0.8)
    ax.set_yticks([])

plt.subplots_adjust(wspace=0.10)
# filename_box_plot = f'{results_boxplots}/results_table_IoU_{len(np.unique(df.image))}_{len(methods)}_{background_type}'

plt.savefig(f'{filename_boxplots}.pdf', dpi=150, transparent=True, bbox_inches='tight', pad_inches=0.02)
# plt.savefig(f'{filename_boxplots}.svg', dpi=150, transparent=True, bbox_inches='tight', pad_inches=0.02)
# plt.savefig(f'{filename_box_plot}.png', dpi=150, transparent=True, bbox_inches='tight', pad_inches=0.02)


plt.show()
# print('file saved at :', filename_box_plot)

# Save Legend

In [ ]:
plot_save_legend = True

## LEGEND FOR E1


In [ ]:
if plot_save_legend:
    legend_order = [
                    ('BPT-100',  0),  ('BPT-500',    2),  ('BPT-1000', 4),
                    ('AA-100',   1),  ('AA-500',     3),   ('AA-1000', 5),
                    ('LIME-100',  6),  ('LIME-500',   7),   ('LIME-1000', 8),
                    # ('LRP',     7),   #12-> 7
                    # ('GradShap', 13), #13 -> 9
                    # ('GradCAM', 9),  ('aIDG',      11),   ('aGradExpl', 12),
                    ]
    lm = [None] * len(legend_order)
    
    for m,j in legend_order:
        lm[j] = m
        print(m,' -> ', j)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

if plot_save_legend:
    ncol = 5
    handles = []
    for m in lm:
        name = name_remap[m] if m in name_remap else m
        print(name)
        p = mpatches.Patch(facecolor=method_colors[m], label=f'{method_to_shortname[m]}:{name}',
                           edgecolor='blue' if m.startswith('BPT') else 'black', 
                           linewidth=1.5 if m.startswith('BPT') else 0.5)
        handles.append(p)

    # Create a figure only for the legend
    fig = plt.figure(figsize=(8, 1))  # Adjust width & height as needed
    ax = fig.add_subplot(111)
    ax.axis("off")  # Hide axes

    legend = fig.legend(handles=handles, ncol=ncol, loc="center", fancybox=True, shadow=True)
    
    
    # Save the legend separately
    fig.savefig(f'{filename_boxplots}_legend.svg', transparent=True, dpi=150, bbox_inches="tight", pad_inches=0.02)
    fig.savefig(f'{filename_boxplots}_legend.pdf', transparent=True, dpi=150, bbox_inches="tight", pad_inches=0.02)
    
    # plt.close(fig)  # Close the figure to prevent showing an empty plot

print(filename_boxplots)


In [ ]:
if plot_save_legend:
    print(filename_boxplots)
    plt.figure()
    handles = []
    ncols = 5
    
    # ncols = 4
    for m in lm:
        
        name = name_remap[m] if m in name_remap else m
        print(name)
        # print(name, method_colors[m], f'{method_to_shortname[m]}:{name}')
        p = mpatches.Patch(facecolor=method_colors[m], label=f'{method_to_shortname[m]}:{name}',
                           edgecolor='blue' if m.startswith('BPT') else 'black', 
                           linewidth=1.5 if m.startswith('BPT') else 0.5)
        handles.append(p)
    text_middle = f'Total Images: \t\t\t{len(np.unique(df.image))}\n Model : \t{model_name} \n background_type:\t{background_type}'
    plt.text(0.4,0.5, text_middle)
    plt.legend(handles=handles, ncols=ncols, loc='upper left', bbox_to_anchor=(0.0, -0.08), 
               fancybox=True, shadow=True, handleheight=1, handlelength=1)
    plt.savefig(f'{filename_boxplots}_legend.svg', transparent=True, dpi=150, bbox_inches='tight', pad_inches=0.02)
    plt.show()


# ANOVA
Test statistical significance of the generated data.

- H0: All group means are equal.
- H1: At least one group mean is different from the rest, i.e. score populations are different.

In [ ]:
from scipy.stats import f_oneway

print('='*108)
print(f'| {"MODEL_TYPE":12} | {"PRETRAINED_MODEL":18} | {"BACKGROUND_TYPE":16} | {"Experiment":15} | {"num_images":<15} | {"num_methods":<13} |')

# print(f'| {model_type:12} | {pretrained_model_type:18} | {background_type:16} | {file_name:15} | {num_images:<15} | {num_methods:<13} |')
print('='*108)
# print(f'| {"-"*20:20} | {"-"*25:25} | {"-"*25:25} | {"-"*25:25} |')
# print('='*108)

print(f'| {"METRIC":<20} | {"SIGNIFICANCE":<25} | {"PVALUE":<53} |')
print('-'*108)
methods = sorted(df.method.unique())
for score in auc_methods:
    if score=='time_exp':
        continue
    pops = []
    for m1 in methods:
        pops.append(df[score][df.method==m1].values)
    
    anova = f_oneway(*pops)
    print(f"| {score:<20} | {'H1' if anova.pvalue<0.05 else 'H0':<25} | {anova.pvalue:<53} |")
print('-'*108)

# END